## textmatcher 2.0

In dieser Version wurde die basale Version im Notebook "textmatcher_basic.ipynb" ausgeweitet und dahingehend adaptiert, dass die Ergebnisse weiter verarbeitet werden können. Hierbei lassen wir uns den gesamten Ouput der Analyse (einmal nur die Tupel und einmal auch die Textpassagen) in eine jeweils neu angelegte Textdatei überschreiben.
Die hier umgesetzte Methode iteriert mit Hilfe einer for-Schleife automatisch über die vorhandenen Textdateien im jeweiligen Ordner und speichert den erzeugten Output in der Textdatei **"output_match_results"**.
Dafür importieren wir neben dem **matcher** auch noch **os** (https://docs.python.org/3/library/os.html) and **plt** von **matplotlib.pyplot** (https://matplotlib.org/stable/api/pyplot_summary.html) für die später vorgenommenen Analysen:

In [ ]:
import os
from text_matcher import matcher
import matplotlib.pyplot as plt

In der ersten Version können wir uns nur die Tupel der gefundenen matches mitsamt der vorangestellten Anzahl der matches ausgeben lassen. Dabei lassen wir uns auch immer den Namen der zum Vergleich herangezogen Textdatei angeben, um die Tupel richtig zuordnen zu können:

In [ ]:
# Laden des Dokuments aus dem "Theosophia" Ordner (bleibt konstant dasselbe)
theosophia_text = matcher.Text(open('theosophia/theosophia.txt', encoding='utf-8').read(), 'Theosophia')

# Arcana Pfad definieren
arcana_folder_path = 'arcana'

# Anlegen des Dictionaries für Kapitelnamen und Anzahl der matches
match_counts = {}

#  output file öffnen und append Resultate
with open('output_match_results.txt', 'a', encoding='utf-8') as output_file:
    # Iteriere über alle Dokumente im Arcana Ordner
    for arcana_file_name in os.listdir(arcana_folder_path):
        arcana_file_path = os.path.join(arcana_folder_path, arcana_file_name)
        
        # Sicherstellung der Arbeit nur mit files
        if os.path.isfile(arcana_file_path):
            # Laden des derzeitigen Arcana files
            arcana_text = matcher.Text(open(arcana_file_path, encoding='utf-8').read(), arcana_file_name)
            
            # match anwenden
            result = matcher.Matcher(theosophia_text, arcana_text).match()
            
            # Nummer der matches extrahieren (erste Nummer im result tuple)
            num_matches = result[0]
            
            # result in dictionary speichern mit Kapitelname als key
            match_counts[arcana_file_name] = num_matches
            
            # result in output file schreiben
            output_file.write(f"Comparing Theosophia with {arcana_file_name}:\n")
            output_file.write(str(result) + "\n\n")

Darauf aufbauend lassen wir uns die entstandenen Ergbenisse mittels **plt** visualisieren. Hierbei stellen wir die Anzahl der gefundenen matches pro Kapitel als Säulendiagramm dar:

In [ ]:
# Plotting von results
chapters = [os.path.splitext(chapter)[0] for chapter in match_counts.keys()]
matches = list(match_counts.values())

plt.figure(figsize=(8, 6), facecolor='white')
ax = plt.gca()  # aktuelle axes adressieren um properties zu ändern
ax.set_facecolor('white')  # Hintergrundfarbe weiß für passende Darstellung

plt.bar(chapters, matches, color='skyblue')
plt.xlabel('Chapters of the Arcana')
plt.ylabel('Number of Matches')
plt.title('Number of Matches per Chapter with the Theosophia')
plt.xticks(rotation=90, ha='right')  # rotieren der chapter names für bessere Lesbarkeit
plt.tight_layout()  # Layout-Anpassung

plt.savefig('match_counts_graph.png', format='png', dpi=100, facecolor='white')

plt.show()

Als Nächstes sollen zunächst anstelle der Tupel die übereinstimmenden Textpassagen in die Textdatei geschrieben werden. Anschließend daran wird die Länge der gegebenen und vorgefundenen Matches mit der Länge der jeweiligen Kapitel der Arcana verglichen und in einem Graph dargestellt. Daran wird ersichtlich welch großer oder kleiner Anteil des jeweiligen Kapitels ebenso in der Theosophia vorkommt.

Nochmals der gesamte Code:

In [ ]:
# Laden des Dokuments aus dem "Theosophia" Ordner (bleibt konstant dasselbe)
theosophia_raw_text = open('theosophia/theosophia.txt', encoding='utf-8').read()
theosophia_text = matcher.Text(theosophia_raw_text, 'Theosophia')

# Arcana Pfad definieren
arcana_folder_path = 'arcana'

# Anlegen der beiden Dictionaries für match lengths und Kapitellänge
match_lengths = {}
chapter_lengths = {}

# output file öffnen und append Resultate
with open('text_output_match_results.txt', 'a', encoding='utf-8') as output_file:
    # Iteriere über alle Dokumente im Arcana Ordner
    for arcana_file_name in os.listdir(arcana_folder_path):
        arcana_file_path = os.path.join(arcana_folder_path, arcana_file_name)
        
        # Sicherstellung der Arbeit nur mit files
        if os.path.isfile(arcana_file_path):
            # Laden des derzeitigen Arcana files
            arcana_raw_text = open(arcana_file_path, encoding='utf-8').read()
            arcana_text = matcher.Text(arcana_raw_text, arcana_file_name)
            
            # match anwenden
            result = matcher.Matcher(theosophia_text, arcana_text).match()
            
            # Unpack result
            match_count, theosophia_matches, arcana_matches = result
            
            # folgenden header für Vergleich in output file schreiben
            output_file.write(f"Comparing Theosophia with {arcana_file_name}:\n")
            output_file.write(f"Number of Matches: {match_count}\n\n")
            
            # length counters
            total_match_length = 0
            total_chapter_length = len(arcana_raw_text)  # Länge des gesamten Kapitels
            
            # über matches iterieren
            for theosophia_match, arcana_match in zip(theosophia_matches, arcana_matches):
                start_theosophia, end_theosophia = theosophia_match
                start_arcana, end_arcana = arcana_match
                
                # Extrahieren des matched text von beiden Texten
                matched_text_theosophia = theosophia_raw_text[start_theosophia:end_theosophia]
                matched_text_arcana = arcana_raw_text[start_arcana:end_arcana]
                
                # Länge berechnen
                match_length = len(matched_text_arcana)  # Länge des matched text
                total_match_length += match_length
                
                # matched text passages in output file schreiben
                output_file.write("Matched passage in Theosophia:\n")
                output_file.write(matched_text_theosophia + "\n\n")
                output_file.write(f"Matched passage in {arcana_file_name}:\n")
                output_file.write(matched_text_arcana + "\n\n")
            
            # beiden errechneten lengths speichern
            match_lengths[arcana_file_name] = total_match_length
            chapter_lengths[arcana_file_name] = total_chapter_length
            
            output_file.write("\n" + "="*50 + "\n\n")  # Verbesserung der Lesbarkeit (Seperator)

# Ergebnisse darstellen/visualisieren
chapters = [os.path.splitext(chapter)[0] for chapter in match_counts.keys()] #list(chapter_lengths.keys())
match_lengths_values = list(match_lengths.values())
chapter_lengths_values = list(chapter_lengths.values())

plt.figure(figsize=(8, 6), facecolor='white')
x = range(len(chapters))
ax = plt.gca()  # aktuelle axes adressieren um properties zu ändern
ax.set_facecolor('white')  # Hintergrundfarbe weiß für passende Darstellung

# bar plot für matched lengths
plt.bar(x, match_lengths_values, width=0.4, label='Length of Matches', color='skyblue', align='center')
# bar plot für chapter lengths
plt.bar([p + 0.4 for p in x], chapter_lengths_values, width=0.4, label='Total Chapter Length', color='lightcoral', align='center')

plt.xlabel('Chapters of the Arcana')
plt.ylabel('Length (Characters)')
plt.title('Length of Matches vs. Total Chapter Length')
plt.xticks([p + 0.2 for p in x], chapters, rotation=45, ha='right')
plt.legend()
plt.tight_layout()
plt.savefig('match_lengths_comparison.png', format='png', dpi=100, facecolor='white')
plt.show()